# MR-KG Backend API Demonstration

This notebook demonstrates the functionality of the MR-KG backend API endpoints.

## Prerequisites

1. Start the backend development server:
   ```bash
   cd backend
   just dev
   ```

2. Ensure the databases are properly configured (see `@docs/ENV.md`)

## Base URL

- Development: http://localhost:8000
- API Documentation: http://localhost:8000/docs

In [ ]:
import requests
import json
from typing import Any
from pprint import pprint

BASE_URL = "http://localhost:8000/api/v1"

def pretty_print(data: Any) -> None:
    """Pretty print JSON data."""
    print(json.dumps(data, indent=2))

def check_response(response: requests.Response) -> dict:
    """Check response status and return JSON data."""
    print(f"Status: {response.status_code}")
    if response.ok:
        return response.json()
    else:
        print(f"Error: {response.text}")
        return {}

## 1. Health Check Endpoints

Monitor system health and diagnostics.

### 1.1 Basic Health Check

In [ ]:
response = requests.get(f"{BASE_URL}/health/")
data = check_response(response)
pretty_print(data)

### 1.2 Detailed Health Check

In [ ]:
response = requests.get(f"{BASE_URL}/health/detailed")
data = check_response(response)
pretty_print(data)

### 1.3 Database Health Check

In [ ]:
response = requests.get(f"{BASE_URL}/health/database")
data = check_response(response)
pretty_print(data)

### 1.4 System Information

In [ ]:
response = requests.get(f"{BASE_URL}/health/system")
data = check_response(response)
pretty_print(data)

### 1.5 Application Metrics

In [ ]:
response = requests.get(f"{BASE_URL}/health/metrics")
data = check_response(response)
pretty_print(data)

### 1.6 Readiness and Liveness Probes

In [ ]:
print("Readiness Check:")
response = requests.get(f"{BASE_URL}/health/ready")
data = check_response(response)
pretty_print(data)

print("\nLiveness Check:")
response = requests.get(f"{BASE_URL}/health/live")
data = check_response(response)
pretty_print(data)

## 2. Traits Endpoints

Explore and analyze traits in the MR-KG database.

### 2.1 List Traits

In [ ]:
response = requests.get(
    f"{BASE_URL}/traits/",
    params={
        "page": 1,
        "page_size": 10,
        "order_by": "appearance_count",
        "order_desc": True
    }
)
data = check_response(response)
pretty_print(data)

### 2.2 Search Traits

In [ ]:
response = requests.get(
    f"{BASE_URL}/traits/search",
    params={
        "q": "diabetes",
        "page": 1,
        "page_size": 10
    }
)
data = check_response(response)
pretty_print(data)

if data.get('data'):
    print(f"\nFound {len(data['data'])} traits matching 'diabetes'")

### 2.3 Get Trait Details

Get comprehensive information about a specific trait.

In [ ]:
trait_index = 1

response = requests.get(
    f"{BASE_URL}/traits/{trait_index}",
    params={
        "include_studies": True,
        "include_similar": True,
        "include_efo": True,
        "max_studies": 5,
        "max_similar": 5,
        "similarity_threshold": 0.5
    }
)
data = check_response(response)

if data.get('data'):
    trait_data = data['data']
    print(f"Trait: {trait_data['trait']['trait_label']}")
    print(f"\nStatistics:")
    pretty_print(trait_data.get('statistics', {}))
    print(f"\nNumber of studies: {len(trait_data.get('studies', []))}")
    print(f"Number of similar traits: {len(trait_data.get('similar_traits', []))}")
    print(f"Number of EFO mappings: {len(trait_data.get('efo_mappings', []))}")

### 2.4 Get Trait Studies

In [ ]:
trait_index = 1

response = requests.get(
    f"{BASE_URL}/traits/{trait_index}/studies",
    params={
        "page": 1,
        "page_size": 5
    }
)
data = check_response(response)

if data.get('data'):
    print(f"Studies for trait {trait_index}:")
    for study in data['data']:
        print(f"\n- PMID: {study['pmid']}")
        print(f"  Model: {study['model']}")
        print(f"  Title: {study.get('title', 'N/A')}")
        print(f"  Journal: {study.get('journal', 'N/A')}")

### 2.5 Find Similar Traits

In [ ]:
trait_index = 1

response = requests.get(
    f"{BASE_URL}/traits/{trait_index}/similar",
    params={
        "max_results": 10,
        "similarity_threshold": 0.3
    }
)
data = check_response(response)

if data.get('data'):
    print(f"Similar traits to trait {trait_index}:")
    for item in data['data'][:5]:
        print(f"\n- Trait Index: {item['result_index']}")
        print(f"  Label: {item['result_label']}")
        print(f"  Similarity: {item['similarity_score']:.4f}")

### 2.6 Get EFO Mappings

In [ ]:
trait_index = 1

response = requests.get(
    f"{BASE_URL}/traits/{trait_index}/efo-mappings",
    params={
        "max_results": 5,
        "similarity_threshold": 0.3
    }
)
data = check_response(response)

if data.get('data'):
    print(f"EFO mappings for trait {trait_index}:")
    for item in data['data']:
        print(f"\n- EFO ID: {item['result_index']}")
        print(f"  Label: {item['result_label']}")
        print(f"  Similarity: {item['similarity_score']:.4f}")

### 2.7 Traits Overview Statistics

In [ ]:
response = requests.get(f"{BASE_URL}/traits/stats/overview")
data = check_response(response)

if data.get('data'):
    overview = data['data']
    print(f"Total traits: {overview['total_traits']}")
    print(f"Total appearances: {overview['total_appearances']}")
    print(f"Average appearances: {overview['average_appearances']}")
    print(f"\nTop 5 traits:")
    for trait in overview['top_traits'][:5]:
        print(f"- {trait['trait_label']}: {trait['appearance_count']} appearances")

### 2.8 Bulk Trait Retrieval

In [ ]:
trait_indices = [1, 2, 3, 4, 5]

response = requests.post(
    f"{BASE_URL}/traits/bulk",
    json=trait_indices
)
data = check_response(response)

if data.get('data'):
    print(f"Retrieved {len(data['data'])} traits:")
    for trait in data['data']:
        print(f"- Index {trait['trait_index']}: {trait['trait_label']}")

## 3. Studies Endpoints

Explore MR studies and their metadata.

### 3.1 List Studies

In [ ]:
response = requests.get(
    f"{BASE_URL}/studies/",
    params={
        "page": 1,
        "page_size": 10,
        "order_by": "pub_date",
        "order_desc": True
    }
)
data = check_response(response)

if data.get('data'):
    print(f"Retrieved {len(data['data'])} studies:")
    for study in data['data'][:5]:
        print(f"\n- ID: {study['id']}")
        print(f"  PMID: {study['pmid']}")
        print(f"  Model: {study['model']}")
        print(f"  Title: {study.get('title', 'N/A')[:80]}...")
        print(f"  Traits: {study['trait_count']}")

### 3.2 Search Studies

In [ ]:
response = requests.get(
    f"{BASE_URL}/studies/search",
    params={
        "q": "cardiovascular",
        "page": 1,
        "page_size": 5
    }
)
data = check_response(response)

if data.get('data'):
    print(f"Found {len(data['data'])} studies matching 'cardiovascular':")
    for study in data['data']:
        print(f"\n- PMID: {study['pmid']}")
        print(f"  Title: {study.get('title', 'N/A')[:80]}...")

### 3.3 Get Study Details

In [ ]:
study_id = 1

response = requests.get(
    f"{BASE_URL}/studies/{study_id}",
    params={
        "include_traits": True,
        "include_similar": True,
        "max_similar": 5
    }
)
data = check_response(response)

if data.get('data'):
    study_data = data['data']
    study_info = study_data['study']
    print(f"Study ID: {study_info['id']}")
    print(f"PMID: {study_info['pmid']}")
    print(f"Model: {study_info['model']}")
    if study_data.get('pubmed_data'):
        pubmed = study_data['pubmed_data']
        print(f"\nTitle: {pubmed.get('title', 'N/A')}")
        print(f"Journal: {pubmed.get('journal', 'N/A')}")
        print(f"Publication Date: {pubmed.get('pub_date', 'N/A')}")
    print(f"\nTraits: {len(study_data.get('traits', []))}")
    print(f"Similar studies: {len(study_data.get('similar_studies', []))}")

### 3.4 Studies Overview Statistics

In [ ]:
response = requests.get(f"{BASE_URL}/studies/stats/overview")
data = check_response(response)

if data.get('data'):
    overview = data['data']
    print(f"Total studies: {overview['total_studies']}")
    print(f"Unique PMIDs: {overview['unique_pmids']}")
    print(f"\nModel distribution:")
    for model, count in list(overview['model_distribution'].items())[:5]:
        print(f"- {model}: {count}")

## 4. Similarity Endpoints

Compute and analyze trait similarities.

### 4.1 Analyze Similarity for PMID-Model Combination

In [ ]:
response = requests.get(
    f"{BASE_URL}/similarities/analyze",
    params={
        "pmid": "12345678",
        "model": "IVW",
        "max_results": 10,
        "min_similarity": 0.3,
        "similarity_type": "trait_profile"
    }
)
data = check_response(response)

if data.get('data'):
    analysis = data['data']
    print("Similarity Analysis:")
    print(f"Query: PMID {analysis['query_combination']['pmid']}, "
          f"Model {analysis['query_combination']['model']}")
    print(f"\nFound {len(analysis['similarities'])} similar combinations")
    pretty_print(analysis.get('summary', {}))

### 4.2 Vector Similarity Search

In [ ]:
trait_index = 1

response = requests.post(
    f"{BASE_URL}/similarities/vector-search",
    json={
        "source_index": trait_index,
        "search_type": "trait",
        "max_results": 10,
        "min_similarity": 0.3
    }
)
data = check_response(response)

if data.get('data'):
    print(f"Vector similarity results for trait {trait_index}:")
    for result in data['data'][:5]:
        print(f"\n- Index: {result['result_index']}")
        print(f"  Label: {result['result_label']}")
        print(f"  Similarity: {result['similarity_score']:.4f}")

### 4.3 Batch Trait-to-EFO Mappings

In [ ]:
trait_indices = [1, 2, 3]

response = requests.post(
    f"{BASE_URL}/similarities/batch-trait-to-efo",
    json={
        "trait_indices": trait_indices,
        "top_k": 3,
        "threshold": 0.3
    }
)
data = check_response(response)

if data.get('data'):
    print("Batch EFO mappings:")
    for mapping in data['data']:
        print(f"\nTrait {mapping['trait_index']}: {mapping['trait_label']}")
        print(f"  EFO mappings: {len(mapping['efo_mappings'])}")
        for efo in mapping['efo_mappings'][:2]:
            print(f"  - {efo['result_label']}: {efo['similarity_score']:.4f}")

## 5. System Endpoints

System-level operations and statistics.

### 5.1 Database Statistics

In [ ]:
response = requests.get(f"{BASE_URL}/system/database/stats")
data = check_response(response)

if data.get('data'):
    stats = data['data']
    print("Database Statistics:")
    pretty_print(stats)

### 5.2 Schema Information

In [ ]:
response = requests.get(f"{BASE_URL}/system/database/schema")
data = check_response(response)

if data.get('data'):
    schema = data['data']
    print("Database Schema:")
    print(f"\nVector Store Tables: {len(schema.get('vector_store', {}).get('tables', []))}")
    print(f"Trait Profile Tables: {len(schema.get('trait_profile', {}).get('tables', []))}")

## 6. Advanced Use Cases

### 6.1 Find Traits and Their Similar Studies

In [ ]:
search_term = "blood pressure"

response = requests.get(
    f"{BASE_URL}/traits/search",
    params={"q": search_term, "page": 1, "page_size": 3}
)
search_data = check_response(response)

if search_data.get('data'):
    print(f"Search results for '{search_term}':")
    for trait in search_data['data']:
        trait_index = trait['trait_index']
        print(f"\n\nTrait: {trait['trait_label']} (index: {trait_index})")
        print(f"Appearances: {trait['appearance_count']}")
        
        studies_response = requests.get(
            f"{BASE_URL}/traits/{trait_index}/studies",
            params={"page": 1, "page_size": 3}
        )
        studies_data = check_response(studies_response)
        
        if studies_data.get('data'):
            print(f"  Related studies: {studies_data['pagination']['total_items']}")
            for study in studies_data['data'][:2]:
                print(f"  - PMID {study['pmid']}: {study.get('title', 'N/A')[:60]}...")

### 6.2 Compare Traits Using Similarity Metrics

In [ ]:
trait_index_1 = 1
trait_index_2 = 2

print(f"Comparing traits {trait_index_1} and {trait_index_2}:\n")

for trait_idx in [trait_index_1, trait_index_2]:
    response = requests.get(f"{BASE_URL}/traits/{trait_idx}")
    data = check_response(response)
    if data.get('data'):
        trait_info = data['data']['trait']
        print(f"Trait {trait_idx}: {trait_info['trait_label']}")

response = requests.get(
    f"{BASE_URL}/traits/{trait_index_1}/similar",
    params={"max_results": 20, "similarity_threshold": 0.0}
)
similar_data = check_response(response)

if similar_data.get('data'):
    for item in similar_data['data']:
        if item['result_index'] == trait_index_2:
            print(f"\nSimilarity score: {item['similarity_score']:.4f}")
            break
    else:
        print(f"\nTrait {trait_index_2} not found in similar traits")

### 6.3 Explore Study Trait Profiles

In [ ]:
study_id = 1

response = requests.get(
    f"{BASE_URL}/studies/{study_id}",
    params={"include_traits": True}
)
data = check_response(response)

if data.get('data'):
    study_data = data['data']
    study_info = study_data['study']
    
    print(f"Study {study_id} - PMID {study_info['pmid']}")
    print(f"Model: {study_info['model']}")
    
    if study_data.get('pubmed_data'):
        print(f"Title: {study_data['pubmed_data'].get('title', 'N/A')}")
    
    print(f"\nTrait Profile ({len(study_data['traits'])} traits):")
    for trait in study_data['traits'][:10]:
        print(f"- Trait {trait['trait_index']}: {trait['trait_label']} "
              f"(role: {trait['role']})")

## 7. Error Handling Examples

### 7.1 Handle Non-existent Resources

In [ ]:
non_existent_trait = 999999

response = requests.get(f"{BASE_URL}/traits/{non_existent_trait}")
print(f"Status Code: {response.status_code}")
if not response.ok:
    print(f"Error Response:")
    pretty_print(response.json())

### 7.2 Handle Invalid Parameters

In [ ]:
response = requests.get(
    f"{BASE_URL}/traits/",
    params={
        "page": -1,
        "page_size": 10000
    }
)
print(f"Status Code: {response.status_code}")
if not response.ok:
    print(f"Validation Error:")
    pretty_print(response.json())

## Summary

This notebook demonstrated:

1. **Health endpoints** for monitoring system status
2. **Traits endpoints** for exploring trait data, similarities, and EFO mappings
3. **Studies endpoints** for accessing MR study information
4. **Similarity endpoints** for computing trait and study similarities
5. **System endpoints** for database statistics and schema information
6. **Advanced use cases** combining multiple endpoints
7. **Error handling** for robust API interactions

## Next Steps

- Explore the interactive API documentation at http://localhost:8000/docs
- Review the backend README at `@backend/README.md`
- Check the API test suite at `@backend/tests/`
- Read the architecture documentation at `@docs/ARCHITECTURE.md`